## Convolutional Networks

We'll check out how to build a **convolutional network** to classify CIFAR10 images. By using weight sharing - multiple units with the same weights - convolutional layers are able to learn repeated patterns in your data. For example, a unit could learn the pattern for an eye, or a face, or lower level features like edges.


In [1]:
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import copy
from Network import ConvNetBig, ConvNetSmall

In [2]:
def get_train_valid_loader(data_dir='data',
                           batch_size=64,
                           augment=False,
                           random_seed = 1,
                           valid_size=0.02,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)

trainloader, valloader = get_train_valid_loader()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# net = ConvNetBig()
net = ConvNetSmall()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

ConvNetSmall(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
net.to(device)

ConvNetSmall(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [4]:
# from torchsummary import summary
# summary(net, input_size=(3, 32, 32))

In [6]:
def get_accuracy(model, dataloader):
    total = 0
    correct = 0
    for (images, labels) in iter(dataloader):
        images, labels = images.to(device), labels.to(device)
        predicted = model.predict(images)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [12]:
epochs = 5
running_loss = 0
print_every = 20
best_accuracy_val = 0
best_model = None
nb_steps = len(trainloader)
accuracy_val = []
accuracy_tr = []

for e in range(epochs):
    start = time.time()
    
    for step, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if step % print_every == 0: # Test accuracy
            stop = time.time()
            net.eval()
            with torch.no_grad():

                accuracy_tr.append(get_accuracy(net, trainloader))
                accuracy_val.append(get_accuracy(net, valloader))
                
                if accuracy_val[-1] > best_accuracy_val: # Save best model
                    best_model = copy.deepcopy(net)
                    best_accuracy_val = accuracy_val[-1]
                    

                print("Epoch: {}/{}".format(e+1, epochs),
                      "Step: {}/{}".format(step, nb_steps),
                      "Loss: {:.4f}..".format(running_loss/print_every),
                      "Train accuracy: {:.3f}..".format(accuracy_tr[-1]),
                      "Test accuracy: {:.3f}..".format(accuracy_val[-1]),
                      "{:.3f} s/{} steps".format((stop - start), print_every)
                     )
                
            running_loss = 0
            start = time.time()
            
print("Best reached accuracy was {}".format(best_accuracy_val))

Epoch: 1/5 Step: 0/766 Loss: 0.1157.. Train accuracy: 8.643.. Test accuracy: 9.300.. 10.924 s/20 steps
Epoch: 1/5 Step: 20/766 Loss: 2.2651.. Train accuracy: 19.102.. Test accuracy: 18.900.. 0.308 s/20 steps
Epoch: 1/5 Step: 40/766 Loss: 2.0919.. Train accuracy: 24.406.. Test accuracy: 24.700.. 0.322 s/20 steps
Epoch: 1/5 Step: 60/766 Loss: 1.9748.. Train accuracy: 27.239.. Test accuracy: 27.200.. 0.313 s/20 steps
Epoch: 1/5 Step: 80/766 Loss: 1.9300.. Train accuracy: 31.524.. Test accuracy: 32.500.. 0.305 s/20 steps
Epoch: 1/5 Step: 100/766 Loss: 1.8999.. Train accuracy: 32.480.. Test accuracy: 32.800.. 0.339 s/20 steps
Epoch: 1/5 Step: 120/766 Loss: 1.8928.. Train accuracy: 31.620.. Test accuracy: 32.300.. 0.319 s/20 steps
Epoch: 1/5 Step: 140/766 Loss: 1.8193.. Train accuracy: 36.565.. Test accuracy: 34.600.. 0.309 s/20 steps
Epoch: 1/5 Step: 160/766 Loss: 1.7644.. Train accuracy: 36.408.. Test accuracy: 35.700.. 0.319 s/20 steps
Epoch: 1/5 Step: 180/766 Loss: 1.7580.. Train accurac

Epoch: 2/5 Step: 760/766 Loss: 1.2719.. Train accuracy: 55.145.. Test accuracy: 52.500.. 0.234 s/20 steps
Epoch: 3/5 Step: 0/766 Loss: 0.3761.. Train accuracy: 55.700.. Test accuracy: 53.200.. 8.743 s/20 steps
Epoch: 3/5 Step: 20/766 Loss: 1.2482.. Train accuracy: 54.457.. Test accuracy: 52.300.. 0.239 s/20 steps
Epoch: 3/5 Step: 40/766 Loss: 1.2677.. Train accuracy: 54.702.. Test accuracy: 52.600.. 0.233 s/20 steps
Epoch: 3/5 Step: 60/766 Loss: 1.2651.. Train accuracy: 56.102.. Test accuracy: 53.300.. 0.241 s/20 steps
Epoch: 3/5 Step: 80/766 Loss: 1.2672.. Train accuracy: 53.908.. Test accuracy: 50.200.. 0.231 s/20 steps
Epoch: 3/5 Step: 100/766 Loss: 1.2366.. Train accuracy: 56.749.. Test accuracy: 53.500.. 0.236 s/20 steps
Epoch: 3/5 Step: 120/766 Loss: 1.2384.. Train accuracy: 55.978.. Test accuracy: 53.100.. 0.234 s/20 steps
Epoch: 3/5 Step: 140/766 Loss: 1.2533.. Train accuracy: 55.837.. Test accuracy: 51.900.. 0.231 s/20 steps
Epoch: 3/5 Step: 160/766 Loss: 1.2377.. Train accura

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(range(len(accuracy_train)), accuracy_train, label="Train")
plt.plot(range(len(accuracy_train)), accuracy_val, label="Test")
plt.legend()
plt.grid()
plt.show()

Save best trained model.

In [13]:
model_name = "model.ckpt"

In [14]:
torch.save(best_model.state_dict(), model_name)

Test that saved correctly

In [15]:
checkpoint = torch.load(model_name)
net.load_state_dict(checkpoint)

<All keys matched successfully>

In [16]:
get_accuracy(net, trainloader), get_accuracy(net, valloader)

(62.310204081632655, 58.9)